In [1]:
from GlucoCheck.glucoCheck import glucoCheckOps
import pandas as pd
import random
import numpy as np
from tqdm.auto import tqdm

Using TensorFlow backend.


In [2]:

from scipy import stats

import random
import re
from dateutil.parser import parse

import warnings  
warnings.filterwarnings('ignore')

import os


In [3]:
def createGap(df,start,end):
    """
    Creating a Gap
    input:
        start: seed
        end: seed + gap
    output:
        df: dataframe with index => DisplayTime value => GlucoseValues and a gap from start to end (inputs)
    """

    #df = readData()
    l = len(df.index)
    if end>l:
        end = l
        
    for i in range(start,end):
        df['GlucoseValue'][i]=0
    
    return df

In [4]:
#Extract Data
data = pd.read_csv("~/Desktop/NCSA_genomics/Python - notebooks/GlucoCheck/Data/Hall/data_hall_raw.csv")

In [5]:
data = data[data['subjectId']=='1636-69-032']
data = data.reset_index(drop=True)

In [6]:
# data.drop(['subjectId'], axis=1, inplace=True)
# data['Display Time'] = data['Display Time'].apply(lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
# data = data.set_index(['Display Time'], drop=True)
data.to_csv("~/Desktop/original.csv")
data


,subjectId,Display Time,GlucoseValue
0,1636-69-032,1/13/16 12:58,122
1,1636-69-032,1/13/16 13:03,123
2,1636-69-032,1/13/16 13:08,124
3,1636-69-032,1/13/16 13:13,128
4,1636-69-032,1/13/16 13:18,133
...,...,...,...
1778,1636-69-032,1/19/16 17:12,101
1779,1636-69-032,1/19/16 17:17,98
1780,1636-69-032,1/19/16 17:22,101
1781,1636-69-032,1/19/16 17:27,106


In [7]:
# sample = random.sample(range(500, 1200), 100)
# sample
#1704,1704,1704,1704,1128,1160
start_points = pd.read_csv('~/Desktop/NCSA_genomics/Python - notebooks/GlucoCheck/Metrics/gap_start.csv')
gap_5 = start_points['g5'].tolist()
gap_15 = start_points['g15'].tolist()
gap_30 = start_points['g30'].tolist()
gap_50 = start_points['g50'].tolist()

In [8]:
obj = glucoCheckOps()

Object Created!


In [10]:
obj.train()

Epoch 1/20
212849/212849 [==============================] - 743s 3ms/step - loss: 114.1132
Epoch 2/20
212849/212849 [==============================] - 373s 2ms/step - loss: 65.8239
Epoch 3/20
212849/212849 [==============================] - 337s 2ms/step - loss: 63.6894
Epoch 4/20
212849/212849 [==============================] - 340s 2ms/step - loss: 63.2470
Epoch 5/20
212849/212849 [==============================] - 341s 2ms/step - loss: 63.1480
Epoch 6/20
212849/212849 [==============================] - 338s 2ms/step - loss: 62.9050
Epoch 7/20
212849/212849 [==============================] - 351s 2ms/step - loss: 61.9935
Epoch 8/20
212849/212849 [==============================] - 347s 2ms/step - loss: 61.8833
Epoch 9/20
212849/212849 [==============================] - 350s 2ms/step - loss: 61.5372
Epoch 10/20
212849/212849 [==============================] - 343s 2ms/step - loss: 61.8123
Epoch 11/20
212849/212849 [==============================] - 323s 2ms/step - loss: 61.4317
Epoch 1

In [11]:
#for gap size 50
ioa_gap50 = list()
fb_gap50 = list()
mad_gap50 = list()
rmse_gap50 = list()
mape_gap50 = list()

for seed in tqdm(gap_50):
    start = seed
    end = seed+50
    data_with_missing = data.copy()
    data_with_missing = createGap(data_with_missing,start,end)
    #     data_with_missing = data_with_missing.drop(data_with_missing.columns[[0]], axis=1)
    imputed_data = obj.impute(data_with_missing,1)

    # data_with_missing.to_csv("~/Desktop/Gap_50.csv")
    # imputed_data.to_csv("~/Desktop/imputed_50.csv")

    mad = obj.mad(np.asarray(imputed_data['GlucoseValue'][start:end-1].tolist()),np.asarray(data['GlucoseValue'][start:end-1].tolist()))
    ioa = obj.index_agreement(np.asarray(imputed_data['GlucoseValue'][start:end-1].tolist()),np.asarray(data['GlucoseValue'][start:end-1].tolist()))
    fb = obj.fracBias(np.asarray(imputed_data['GlucoseValue'][start:end-1].tolist()),np.asarray(data['GlucoseValue'][start:end-1].tolist()))
    rmse = obj.rmse(np.asarray(imputed_data['GlucoseValue'][start:end-1].tolist()),np.asarray(data['GlucoseValue'][start:end-1].tolist()))
    mape = obj.mape(np.asarray(imputed_data['GlucoseValue'][start:end-1].tolist()),np.asarray(data['GlucoseValue'][start:end-1].tolist()))

    mad_gap50.append(mad)
    ioa_gap50.append(ioa)
    fb_gap50.append(fb)
    rmse_gap50.append(rmse)
    mape_gap50.append(mape)



In [12]:
#for gap size 30
ioa_gap30 = list()
fb_gap30 = list()
mad_gap30 = list()
rmse_gap30 = list()
mape_gap30 = list()

for seed in tqdm(gap_30):
    start = seed
    end = start+30
    data_with_missing = data.copy()
    data_with_missing = createGap(data_with_missing,start,end)
    #     data_with_missing = data_with_missing.drop(data_with_missing.columns[[0]], axis=1)
    imputed_data = obj.impute(data_with_missing,1)

    # data_with_missing.to_csv("~/Desktop/Gap_30.csv")
    # imputed_data.to_csv("~/Desktop/imputed_30.csv")

    mad = obj.mad(np.asarray(imputed_data['GlucoseValue'][start:end-1].tolist()),np.asarray(data['GlucoseValue'][start:end-1].tolist()))
    ioa = obj.index_agreement(np.asarray(imputed_data['GlucoseValue'][start:end-1].tolist()),np.asarray(data['GlucoseValue'][start:end-1].tolist()))
    fb = obj.fracBias(np.asarray(imputed_data['GlucoseValue'][start:end-1].tolist()),np.asarray(data['GlucoseValue'][start:end-1].tolist()))
    rmse = obj.rmse(np.asarray(imputed_data['GlucoseValue'][start:end-1].tolist()),np.asarray(data['GlucoseValue'][start:end-1].tolist()))
    mape = obj.mape(np.asarray(imputed_data['GlucoseValue'][start:end-1].tolist()),np.asarray(data['GlucoseValue'][start:end-1].tolist()))

    mad_gap30.append(mad)
    ioa_gap30.append(ioa)
    fb_gap30.append(fb)
    rmse_gap30.append(rmse)
    mape_gap30.append(mape)



In [13]:
#for gap size 15
ioa_gap15 = list()
fb_gap15 = list()
mad_gap15 = list()
rmse_gap15 = list()
mape_gap15 = list()

for seed in tqdm(gap_15):
    start = seed
    end = start+15
    data_with_missing = data.copy()
    data_with_missing = createGap(data_with_missing,start,end)
    # data_with_missing = data_with_missing.drop(data_with_missing.columns[[0]], axis=1)
    imputed_data = obj.impute(data_with_missing,1)

    # data_with_missing.to_csv("~/Desktop/Gap_15.csv")
    # imputed_data.to_csv("~/Desktop/imputed_15.csv")

    mad = obj.mad(np.asarray(imputed_data['GlucoseValue'][start:end-1].tolist()),np.asarray(data['GlucoseValue'][start:end-1].tolist()))
    ioa = obj.index_agreement(np.asarray(imputed_data['GlucoseValue'][start:end-1].tolist()),np.asarray(data['GlucoseValue'][start:end-1].tolist()))
    fb = obj.fracBias(np.asarray(imputed_data['GlucoseValue'][start:end-1].tolist()),np.asarray(data['GlucoseValue'][start:end-1].tolist()))
    rmse = obj.rmse(np.asarray(imputed_data['GlucoseValue'][start:end-1].tolist()),np.asarray(data['GlucoseValue'][start:end-1].tolist()))
    mape = obj.mape(np.asarray(imputed_data['GlucoseValue'][start:end-1].tolist()),np.asarray(data['GlucoseValue'][start:end-1].tolist()))

    mad_gap15.append(mad)
    ioa_gap15.append(ioa)
    fb_gap15.append(fb)
    rmse_gap15.append(rmse)
    mape_gap15.append(mape)

    

In [14]:
#for gap size 5
ioa_gap5 = list()
fb_gap5 = list()
mad_gap5 = list()
rmse_gap5 = list()
mape_gap5 = list()

for seed in tqdm(gap_5):
    start = seed
    end = start+5
    data_with_missing = data.copy()
    data_with_missing = createGap(data_with_missing,start,end)
    #     data_with_missing = data_with_missing.drop(data_with_missing.columns[[0]], axis=1)
    imputed_data = obj.impute(data_with_missing,1)

    # data_with_missing.to_csv("~/Desktop/Gap_5.csv")
    # imputed_data.to_csv("~/Desktop/imputed_5.csv")

    mad = obj.mad(np.asarray(imputed_data['GlucoseValue'][start:end-1].tolist()),np.asarray(data['GlucoseValue'][start:end-1].tolist()))
    ioa = obj.index_agreement(np.asarray(imputed_data['GlucoseValue'][start:end-1].tolist()),np.asarray(data['GlucoseValue'][start:end-1].tolist()))
    fb = obj.fracBias(np.asarray(imputed_data['GlucoseValue'][start:end-1].tolist()),np.asarray(data['GlucoseValue'][start:end-1].tolist()))
    rmse = obj.rmse(np.asarray(imputed_data['GlucoseValue'][start:end-1].tolist()),np.asarray(data['GlucoseValue'][start:end-1].tolist()))
    mape = obj.mape(np.asarray(imputed_data['GlucoseValue'][start:end-1].tolist()),np.asarray(data['GlucoseValue'][start:end-1].tolist()))

    mad_gap5.append(mad)
    ioa_gap5.append(ioa)
    fb_gap5.append(fb)
    rmse_gap5.append(rmse)
    mape_gap5.append(mape)



In [15]:
MAD = pd.DataFrame({'Gap:5':mad_gap5, 'Gap:15':mad_gap15, 'Gap:30':mad_gap30, 'Gap:50':mad_gap50})
MAD

,Gap:5,Gap:15,Gap:30,Gap:50
0,3.50,10.071429,14.413793,17.000000
1,3.50,21.214286,30.000000,32.897959
2,4.50,7.857143,19.827586,22.918367
3,16.75,28.142857,40.862069,41.877551
4,4.75,16.857143,29.931034,36.306122
...,...,...,...,...
95,10.25,9.214286,26.517241,20.673469
96,3.25,21.928571,21.655172,21.265306
97,1.50,11.357143,13.724138,39.081633
98,11.50,23.428571,31.172414,28.122449


In [16]:
IOA = pd.DataFrame({'Gap:5':ioa_gap5, 'Gap:15':ioa_gap15, 'Gap:30':ioa_gap30, 'Gap:50':ioa_gap50})
IOA


,Gap:5,Gap:15,Gap:30,Gap:50
0,0.301887,0.276361,0.400371,0.401743
1,0.483333,0.231432,0.260191,0.359407
2,0.726150,0.486180,0.245001,0.292173
3,0.244808,0.053866,0.046952,0.302075
4,0.154639,0.094439,0.152901,0.294611
...,...,...,...,...
95,0.225122,0.577678,0.175651,0.532635
96,0.350211,0.199941,0.537754,0.368071
97,0.944882,0.279658,0.440465,0.225042
98,0.152738,0.420421,0.140992,0.236711


In [17]:
FB = pd.DataFrame({'Gap:5':fb_gap5, 'Gap:15':fb_gap15, 'Gap:30':fb_gap30, 'Gap:50':fb_gap50})
FB

,Gap:5,Gap:15,Gap:30,Gap:50
0,0.038283,0.112715,0.174055,0.211889
1,0.031561,0.189653,0.300290,0.355800
2,0.054192,0.100247,0.243024,0.281302
3,0.144585,0.256030,0.400372,0.436724
4,0.042442,0.158152,0.304053,0.386198
...,...,...,...,...
95,0.098092,0.095767,0.284684,0.249609
96,0.035978,0.231320,0.251094,0.262738
97,0.013107,0.141036,0.174946,0.427072
98,0.114257,0.220757,0.316053,0.342574


In [18]:
RMSE = pd.DataFrame({'Gap:5':rmse_gap5, 'Gap:15':rmse_gap15, 'Gap:30':rmse_gap30})
RMSE

,Gap:5,Gap:15,Gap:30
0,4.301163,11.865316,16.339079
1,3.937004,24.886887,32.750888
2,5.522681,9.621405,23.526214
3,17.965244,29.681644,43.277373
4,5.545268,19.563085,33.897411
...,...,...,...
95,10.897247,11.762531,32.221753
96,4.387482,23.187127,22.581653
97,1.870829,14.144661,16.821579
98,12.124356,24.086155,36.386811


In [19]:
MAPE = pd.DataFrame({'Gap:5':mape_gap5, 'Gap:15':mape_gap15, 'Gap:30':mape_gap30, 'Gap:50':mape_gap50})
MAPE

,Gap:5,Gap:15,Gap:30,Gap:50
0,3.721156,10.460735,15.659096,18.744331
1,3.114313,16.802246,25.401667,29.613152
2,5.655908,9.990708,21.166685,24.141868
3,13.359892,22.393983,32.605461,35.262710
4,4.125010,14.288676,25.432642,31.402919
...,...,...,...,...
95,10.389766,8.871233,23.562124,21.892171
96,3.483750,20.461274,22.121743,22.485938
97,1.299638,12.742595,15.554732,34.322903
98,10.745243,19.722439,26.085130,28.899280


In [20]:
IOA.to_csv("~/Desktop/NCSA_genomics/Python - notebooks/GlucoCheck/Metrics/IOA.csv")
FB.to_csv("~/Desktop/NCSA_genomics/Python - notebooks/GlucoCheck/Metrics/FB.csv")
RMSE.to_csv("~/Desktop/NCSA_genomics/Python - notebooks/GlucoCheck/Metrics/RMSE.csv")
MAPE.to_csv("~/Desktop/NCSA_genomics/Python - notebooks/GlucoCheck/Metrics/MAPE.csv")
MAD.to_csv("~/Desktop/NCSA_genomics/Python - notebooks/GlucoCheck/Metrics/MAD.csv")